# Project Introduction

This project aims to build a sopisticated a movie recommendation system using machine learning algorithms. 

We have read in a movies database from kaggle (https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata) which contains two csv files; one of them has information about the casts and crews and the other has information about the movies itself. 

Using both the information in the csv file we will create a machine learning model system that when a user enters a random movie from the database, then the model will recommend five movies similar to that movie to the user.

# Scope of the project

We will create a recommendation system using cosine similarity from sklearn. In this project we will take the important information from the two csv files and merge it into a simgle dataframe. 

We will need to do some feature engineering and feature extraction as some of the important information are given in a text format which can also be converted to a dictionary and its keys can be called to get the desired information.

In this project we will be doing content based recommendation system. In future we can try to do a conditional filtering based recommendation system.

Since we are doing content based recommendation system we will be taking all the important information from the csv files and put it in a single column (also known as creating tags), then do preprocessing as needed and then convert the text into vectors. 

After that using cosine similarity index we will be building a system that recommends five similar type movies when given a single movie.

## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud,STOPWORDS

## Importing data from csv

In [2]:
credits_df  = pd.read_csv("tmdb_5000_credits.csv")
credits_df.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [3]:
credits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [4]:
movies_df  = pd.read_csv("tmdb_5000_movies.csv").drop(['title'],axis =1)
movies_df.rename({'id':'movie_id'},axis =1,inplace =True)
movies_df.head(2)

,budget,genres,homepage,movie_id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",6.9,4500


In [5]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   movie_id              4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

## Merging two dataframes

In [6]:
movies_df = movies_df.merge(credits_df,on = "movie_id")

#### We will keep only the required columns

In [7]:
movies_df = movies_df[['genres','movie_id','keywords','title','production_companies',
          'cast','crew','overview']]

movies_df.head(3)

,genres,movie_id,keywords,title,production_companies,cast,crew,overview
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","In the 22nd century, a paraplegic Marine is di..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",Pirates of the Caribbean: At World's End,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","Captain Barbossa, long believed to be dead, ha..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",Spectre,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",A cryptic message from Bond’s past sends him o...


In [8]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   genres                4803 non-null   object
 1   movie_id              4803 non-null   int64 
 2   keywords              4803 non-null   object
 3   title                 4803 non-null   object
 4   production_companies  4803 non-null   object
 5   cast                  4803 non-null   object
 6   crew                  4803 non-null   object
 7   overview              4800 non-null   object
dtypes: int64(1), object(7)
memory usage: 337.7+ KB


## Dropping null values

In [9]:
movies_df = movies_df.dropna()

## Data Preprocessing

In [10]:
movies_df['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [11]:
movies_df['keywords'][0]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [12]:
movies_df["production_companies"][0]

'[{"name": "Ingenious Film Partners", "id": 289}, {"name": "Twentieth Century Fox Film Corporation", "id": 306}, {"name": "Dune Entertainment", "id": 444}, {"name": "Lightstorm Entertainment", "id": 574}]'

### Data Preprocessing Strategy: "genres" & "keywords" Column

The columns "genres", "keywords" and "production_companies" has a 'name' key that has the required important information about the movies. We need to extract this information by creating a fuction.

#### We will create a function to extract information

In [13]:
import json

def extract_name(string):
    data = json.loads(string)
    names = []
    for item in data:
        names.append(item["name"])
    return names   

#### We will create a function to convert list items to string

In [14]:
def convert_list_to_string(items):
    return ' '.join(items)

In [15]:
cols_to_change = ["keywords","genres","production_companies"]

for i in cols_to_change:
    movies_df[i] = movies_df[i].apply(extract_name)

In [16]:
movies_df.head(2)

,genres,movie_id,keywords,title,production_companies,cast,crew,overview
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"[Ingenious Film Partners, Twentieth Century Fo...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","In the 22nd century, a paraplegic Marine is di..."
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End,"[Walt Disney Pictures, Jerry Bruckheimer Films...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","Captain Barbossa, long believed to be dead, ha..."


### Data Preprocessing Strategy : "Cast" Column

In the cast column there are lots of information about the movies cast. We will take only the first three cast names of the certain movie.

#### We will create a function to get the first three names in "cast" column

In [17]:
import json

def get_first_three_names(json_string):
    data = json.loads(json_string)
    names = []
    for entry in data[:3]:
        names.append(entry['name'])
    return names


In [18]:
movies_df['cast'] = movies_df['cast'].apply(get_first_three_names)


In [19]:
movies_df.head(2)

,genres,movie_id,keywords,title,production_companies,cast,crew,overview
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"[Ingenious Film Partners, Twentieth Century Fo...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","In the 22nd century, a paraplegic Marine is di..."
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End,"[Walt Disney Pictures, Jerry Bruckheimer Films...","[Johnny Depp, Orlando Bloom, Keira Knightley]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","Captain Barbossa, long believed to be dead, ha..."


### Data Preprocessing Strategy : "Crew" Column

From the "crew" column we will extract only the director of the movies are they are very important to most people who watch movies

#### We will create a function to get the director name in "crew" column

In [20]:
import json

def extract_director(string):
    data = json.loads(string)
    names = []
    for item in data:
        if item['job'] == "Director":
            names.append(item["name"])
    return names
        

In [21]:
movies_df['crew'] = movies_df['crew'].apply(extract_director)

In [22]:
movies_df.head(2)

,genres,movie_id,keywords,title,production_companies,cast,crew,overview
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"[Ingenious Film Partners, Twentieth Century Fo...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"In the 22nd century, a paraplegic Marine is di..."
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End,"[Walt Disney Pictures, Jerry Bruckheimer Films...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski],"Captain Barbossa, long believed to be dead, ha..."


### Removing unwanted spaces from columns texts

We need to remove spaces from the preprocessed columns so that the important information of the columns are not lost and its value is retained

#### We will create a function to remove spaces from the text of each column

In [23]:
def remove_spaces_in_list(items):
    return [item.replace(" ", "") for item in items]


In [24]:
cols_to_change = ['production_companies','cast','crew']

for i in cols_to_change:
    movies_df[i] = movies_df[i].apply(remove_spaces_in_list)

In [25]:
movies_df.head(2)

,genres,movie_id,keywords,title,production_companies,cast,crew,overview
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"[IngeniousFilmPartners, TwentiethCenturyFoxFil...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"In the 22nd century, a paraplegic Marine is di..."
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End,"[WaltDisneyPictures, JerryBruckheimerFilms, Se...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"Captain Barbossa, long believed to be dead, ha..."


### Converting the string to list type

In [26]:
def convert_string_to_list(string):
    return string.split()

movies_df["overview"] = movies_df["overview"].apply(convert_string_to_list)

### Creating the tags by combining the extracted features

In [27]:
movies_df['tags'] = movies_df['overview'] + movies_df['genres'] + movies_df['keywords'] + movies_df['cast']+ movies_df['crew']

### Creating a final dataframe with the required information

In [28]:
new_df  = movies_df[['movie_id','title','tags']]

In [29]:
new_df

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...
4798,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui..."
4799,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended..."
4800,231617,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,..."
4801,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


#### We will convert the list items to string

In [30]:
def convert_list_to_string(lst):
    return ' '.join(lst)

In [31]:
new_df['tags'] = new_df['tags'].apply(convert_list_to_string)

C:\Users\antho\AppData\Local\Temp\ipykernel_10936\2057858984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(convert_list_to_string)


In [32]:
new_df['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [33]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


### Feature Engineering Strategy: Clean the "tags" column

#### We will create a function to clean our tags column

In [34]:
def text_processing(data,review):
    digit = '1234567890'
    from bs4 import BeautifulSoup
    import re
    def decontracted(phrase):
        # specific
        phrase= re.sub(r'<br /><br />',' ',phrase)
        phrase = re.sub(r"won't", "will not", phrase)
        phrase = re.sub(r"can\'t", "can not", phrase)

        # general
        phrase = re.sub(r"n\'t", " not", phrase)
        phrase = re.sub(r"\'re", " are", phrase)
        phrase = re.sub(r"\'s", " is", phrase)
        phrase = re.sub(r"\'d", " would", phrase)
        phrase = re.sub(r"\'ll", " will", phrase)
        phrase = re.sub(r"\'t", " not", phrase)
        phrase = re.sub(r"\'ve", " have", phrase)
        phrase = re.sub(r"\'m", " am", phrase)
        phrase = re.sub(r'"', " ", phrase)
        return phrase
    stopwords=set(STOPWORDS)

    # Combining all the above stundents 
    from tqdm import tqdm
    preprocessed_reviews = []
    # tqdm is for printing the status bar
    for sentance in tqdm(data[review].values):
        sentance = re.sub(r"http\S+", "", sentance)
        sentance = BeautifulSoup(sentance, 'lxml').get_text()
        sentance = decontracted(sentance)
        sentance = re.sub("\S*\d\S*", "", sentance).strip()
        sentance = sentance.lower()
        # https://gist.github.com/sebleier/554280
        sentance = ' '.join(e.lower() for e in sentance.split() if e not in stopwords )
        preprocessed_reviews.append(sentance.strip())
    
    new_preprocessed_reviews=[]
    for sentennce in preprocessed_reviews:
        for char in sentennce:
            if char in digit:
                sentennce = sentennce.replace(char,'')
        new_preprocessed_reviews.append(sentennce)
    # removing punctuations   
    import string
    translator = str.maketrans('', '', string.punctuation)
    new_preprocessed_reviews = [sentence.translate(translator) for sentence in new_preprocessed_reviews]
                
    
    from nltk.stem import PorterStemmer

    porter = PorterStemmer()
    list2=[]
    for  sentence in new_preprocessed_reviews:
      list1=[]
      for words in sentence.split():
            list1.append(porter.stem(words))

      list2.append(' '.join(list1))
    return(list2)


In [35]:
new_df['cleaned_tags']=text_processing(new_df,'tags')

 73%|███████████████████████████████████████████████████████▊                    | 3524/4800 [00:00<00:00, 4966.06it/s]C:\Users\antho\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████| 4800/4800 [00:01<00:00, 4772.10it/s]
C:\Users\antho\AppData\Local\Temp\ipykernel_10936\2438430064.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['cleaned_tags']=text_processing(new_df,'tags')


In [36]:
new_df.head(2)

,movie_id,title,tags,cleaned_tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",centuri parapleg marin dispatch moon pandora u...
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",captain barbossa long believ dead come back li...


In [37]:
new_df["tags"][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [38]:
new_df["cleaned_tags"][0]

'centuri parapleg marin dispatch moon pandora uniqu mission becom torn follow order protect alien civil action adventur fantasi scienc fiction cultur clash futur space war space coloni societi space travel futurist romanc space alien tribe alien planet cgi marin soldier battl love affair anti war power relat mind soul samworthington zoesaldana sigourneyweav jamescameron'

### Converting the string column into vectors

In [39]:
from sklearn.feature_extraction.text import CountVectorizer
# Vectorize the text data
count_vect = CountVectorizer(max_features=5000, stop_words='english')
cleaned_tags_vectorized = count_vect.fit_transform(new_df["cleaned_tags"]).toarray()

In [40]:
cleaned_tags_vectorized.shape

(4800, 5000)

In [41]:
cleaned_tags_vectorized[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [42]:
count_vect.get_feature_names()[:20]

C:\Users\antho\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['aaron',
 'aaroneckhart',
 'abandon',
 'abbi',
 'abduct',
 'abigailbreslin',
 'abil',
 'abl',
 'aboard',
 'aborigin',
 'abroad',
 'absenc',
 'abus',
 'academ',
 'academi',
 'accent',
 'accept',
 'access',
 'accid',
 'accident']

### Cosine similarity to calculate the similarity between vectors

In [43]:
from sklearn.metrics.pairwise import cosine_similarity

In [44]:
similarity = cosine_similarity(cleaned_tags_vectorized)

In [45]:
similarity.shape

(4800, 4800)

### Building the recommendation system

We will create a function that will take a movie as input, then find the index postion of the movie then find cosine similarity of other movies, with respect to the inputed movie; and then sort the listed movies so that the top five highest similar movies is fetched and shown to the user.

In [46]:
def recommend(movie):
    try:
        
        # find the index position of the selected movie
        movie_index = new_df[new_df['title'] == movie].index[0]
        
        # calculate distances
        distances = similarity[movie_index]
        
        # create a sorted movies list that fetches top 5 similar movies
        movies_list = sorted(list(enumerate(distances)),reverse=True,key = lambda x:x[1])[1:6]
        
        # show the movie title to the user
        for i in movies_list:
            print(new_df.iloc[i[0]].title)
            
    # what happens if a user inputs movie that is not in database       
    except:
        print("That movie is not in our database")

### Testing our recommendation system

In [47]:
recommend('Batman')

Batman & Robin
Batman Begins
The Dark Knight Rises
Batman Returns
The Dark Knight


In [48]:
recommend('Harry Potter and the Half-Blood Prince')

Harry Potter and the Goblet of Fire
Harry Potter and the Philosopher's Stone
Harry Potter and the Order of the Phoenix
Harry Potter and the Prisoner of Azkaban
Harry Potter and the Chamber of Secrets


In [49]:
recommend('Man of Steel')

Superman III
Batman v Superman: Dawn of Justice
Superman
Superman Returns
Superman II


In [50]:
recommend("Pirates of the Caribbean: Dead Man's Chest")

Pirates of the Caribbean: At World's End
Pirates of the Caribbean: The Curse of the Black Pearl
Pirates of the Caribbean: On Stranger Tides
The Pirates! In an Adventure with Scientists!
Captain Phillips


In [51]:
recommend("Quantum of Solace")

Spectre
Never Say Never Again
Diamonds Are Forever
From Russia with Love
Octopussy


In [52]:
recommend('Harry Potter')

That movie is not in our database


# Saving Model for deployment

In [53]:
import pickle

pickle.dump(new_df.to_dict(), open('movie_dict.pkl','wb'))

In [54]:
pickle.dump(similarity,open('similarity.pkl','wb'))

# Conclusion

Our recommedation system seems to be working as desired. 

During our testing we found almost 100 percent accuracy when predicting a similar type of movie. 

We recommend creating a system using conditional filtering and see if that system is more accurate then the one presented here.